# Capsule Network for RL

In [1]:
import tensorflow as tf
import cv2
import sys
import time 
sys.path.append("game/")
import wrapped_flappy_bird as game
import random
import numpy as np
from collections import deque
from matplotlib import pyplot as plt

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
epsilon = 1e-9
iter_routing = 2
train_freq = 10

In [3]:
GAME = 'bird' # the name of the game being played for log files
ACTIONS = 2 # number of valid actions
GAMMA = 0.99 # decay rate of past observations
OBSERVE = 1000 # timesteps to observe before training
EXPLORE = 300000 # frames over which to anneal epsilon
FINAL_EPSILON = 0.0001 # final value of epsilon
INITIAL_EPSILON = 0.1 # starting value of epsilon
REPLAY_MEMORY = 50000 # number of previous transitions to remember
BATCH = 32 # size of minibatch

# UPDATED CAPSULE NETWORK

In [4]:
def squash(vector):
    vec_squared_norm = reduce_sum(tf.square(vector), -2, keepdims=True)
    scalar_factor = vec_squared_norm / (1 + vec_squared_norm) / tf.sqrt(vec_squared_norm + epsilon)
    vec_squashed = scalar_factor * vector  # element-wise
    return(vec_squashed)
def routing(input, b_IJ):
    # W: [1, num_caps_i, num_caps_j * len_v_j, len_u_j, 1]
    W = tf.get_variable('Weight', shape=(1, 1024, 160, 8, 1), dtype=tf.float32,
                        initializer=tf.random_normal_initializer(stddev=0.01))
    biases = tf.get_variable('bias', shape=(1, 1, 10, 16, 1))
    # A better solution is using element-wise multiply, reduce_sum and reshape
    # ops instead. Matmul [a, b] x [b, c] is equal to a series ops as
    # element-wise multiply [a*c, b] * [a*c, b], reduce_sum at axis=1 and
    # reshape to [a, c]
    input = tf.tile(input, [1, 1, 160, 1, 1])
    #assert input.get_shape() == [cfg.batch_size, 1024, 160, 8, 1]

    u_hat = reduce_sum(W * input, axis=3, keepdims=True)
    u_hat = tf.reshape(u_hat, shape=[-1, 1024, 10, 16, 1])
    #assert u_hat.get_shape() == [cfg.batch_size, 1024, 10, 16, 1]

    # In forward, u_hat_stopped = u_hat; in backward, no gradient passed back from u_hat_stopped to u_hat
    u_hat_stopped = tf.stop_gradient(u_hat, name='stop_gradient')

    # line 3,for r iterations do
    for r_iter in range(iter_routing):
        with tf.variable_scope('iter_' + str(r_iter)):
            # line 4:
            # => [batch_size, 1024, 10, 1, 1]
            c_IJ = softmax(b_IJ, axis=2)

            # At last iteration, use `u_hat` in order to receive gradients from the following graph
            if r_iter == iter_routing - 1:
                # line 5:
                # weighting u_hat with c_IJ, element-wise in the last two dims
                # => [batch_size, 1024, 10, 16, 1]
                s_J = tf.multiply(c_IJ, u_hat)
                # then sum in the second dim, resulting in [batch_size, 1, 10, 16, 1]
                s_J = reduce_sum(s_J, axis=1, keepdims=True) + biases
                #assert s_J.get_shape() == [cfg.batch_size, 1, 10, 16, 1]

                # line 6:
                # squash using Eq.1,
                v_J = squash(s_J)
                #assert v_J.get_shape() == [cfg.batch_size, 1, 10, 16, 1]
            elif r_iter < iter_routing - 1:  # Inner iterations, do not apply backpropagation
                s_J = tf.multiply(c_IJ, u_hat_stopped)
                s_J = reduce_sum(s_J, axis=1, keepdims=True) + biases
                v_J = squash(s_J)

                # line 7:
                # reshape & tile v_j from [batch_size ,1, 10, 16, 1] to [batch_size, 1024, 10, 16, 1]
                # then matmul in the last tow dim: [16, 1].T x [16, 1] => [1, 1], reduce mean in the
                # batch_size dim, resulting in [1, 1024, 10, 1, 1]
                v_J_tiled = tf.tile(v_J, [1, 1024, 1, 1, 1])
                u_produce_v = reduce_sum(u_hat_stopped * v_J_tiled, axis=3, keepdims=True)
                #assert u_produce_v.get_shape() == [cfg.batch_size, 1024, 10, 1, 1]

                # b_IJ += tf.reduce_sum(u_produce_v, axis=0, keep_dims=True)
                b_IJ += u_produce_v
    return(v_J)
# For version compatibility
def reduce_sum(input_tensor, axis=None, keepdims=False):
    return tf.reduce_sum(input_tensor, axis=axis, keepdims=keepdims)
# For version compatibility
def softmax(logits, axis=None):
    return tf.nn.softmax(logits, axis=axis)

In [5]:
def createNetwork():
    # input layer
    s= tf.placeholder("float", [None, 84, 84, 4])
    coeff = tf.placeholder(tf.float32, shape=(None, 1024, 10, 1, 1))
    ####################### New Network COnfiguration #####################    
    w_initializer, b_initializer = tf.random_normal_initializer(0., 0.01), tf.constant_initializer(0.01)
    w1 = tf.get_variable('w1',[8, 8, 4, 64],initializer=w_initializer)
    b1 = tf.get_variable('b1',[64],initializer=b_initializer)
    # Convolution Layer
    # Conv1, [batch_size, 20, 20, 64]
    l1 = tf.nn.conv2d(s, w1, strides=[1, 4, 4, 1], padding="VALID")
    
    conv1 = tf.nn.relu(tf.nn.bias_add(l1, b1))
    
    conv1 = tf.reshape(conv1,[-1,20,20,64])
    
    capsules = tf.contrib.layers.conv2d(conv1, 16 * 8, kernel_size=6, stride=2, padding="VALID",
                    activation_fn = tf.nn.relu,
                    weights_initializer = tf.contrib.layers.xavier_initializer(uniform=False),
                    biases_initializer=tf.constant_initializer(0))
    
    capsules = tf.reshape(capsules, (-1, 1024, 8, 1)) #Reshape to(batch_szie, 1152, 8, 1)
    
    capsules = squash(capsules)
    
    input_caps2 = tf.reshape(capsules, shape=(-1, 1024, 1, capsules.shape[-2].value, 1))
    
    caps2 = routing(input_caps2, coeff)
    
    vector_j = tf.reshape(caps2, shape=(-1, 160))
    #print(vector_j)
    q_eval = tf.contrib.layers.fully_connected(vector_j, num_outputs=ACTIONS, activation_fn=None)

    #print(q_eval)
    readout = q_eval
    return s, coeff, readout

In [6]:
def trainNetwork(s, coeff, readout, sess):
    tick = time.time()
    # define the cost function
    a = tf.placeholder("float", [None, ACTIONS])
    y = tf.placeholder("float", [None])
    readout_action = tf.reduce_sum(tf.multiply(readout, a), reduction_indices = 1)
    cost = tf.reduce_mean(tf.square(y - readout_action))
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cost)

    # open up a game state to communicate with emulator
    game_state = game.GameState()
    
    # store the previous observations in replay memory
    D = deque()
    
    # get the first state by doing nothing and preprocess the image to 84x84x4
    do_nothing = np.zeros(ACTIONS)
    do_nothing[0] = 1
    x_t, r_0, terminal = game_state.frame_step(do_nothing)
    x_t = cv2.cvtColor(cv2.resize(x_t[:,:-110,:], (84, 84)), cv2.COLOR_BGR2GRAY)
    ret, x_t = cv2.threshold(x_t,1,255,cv2.THRESH_BINARY)
    s_t = np.stack((x_t, x_t, x_t, x_t), axis=2)
    sess.run(tf.global_variables_initializer())
    # saving and loading networks
    saver = tf.train.Saver()
    checkpoint = tf.train.get_checkpoint_state("saved_networks")
    """if checkpoint and checkpoint.model_checkpoint_path:
        saver.restore(sess, checkpoint.model_checkpoint_path)
        print("Successfully loaded:", checkpoint.model_checkpoint_path)
    else:
        print("Could not find old network weights")"""
        
    sess.run(tf.global_variables_initializer())
    b_IJ1 = np.zeros((1, 1024, 10, 1, 1)).astype(np.float32) # batch_size=1
    b_IJ2 = np.zeros((BATCH, 1024, 10, 1, 1)).astype(np.float32) # batch_size=BATCH
    epsilon = INITIAL_EPSILON
    t = 0
    pscore = 0
    episode = 0
    loss = 0
    Q_MAX = -1100000
    tick = time.time()
    action_freq = np.zeros(ACTIONS)
    while True:
        # choose an action epsilon greedily
        # readout_t = readout.eval(feed_dict = {s : [s_t].reshape((1,80,80,4))})[0]
        
        readout_t = readout.eval(feed_dict = {s:s_t.reshape((1,84,84,4)), coeff:b_IJ1})
        
        a_t = np.zeros([ACTIONS])
        action_index = 0
        if random.random() <= epsilon or t <= OBSERVE:
            action_index = random.randrange(ACTIONS)
            a_t[action_index] = 1
        else:
            action_index = np.argmax(readout_t)
            a_t[action_index] = 1

        # scale down epsilon
        if epsilon > FINAL_EPSILON and t > OBSERVE:
            epsilon -= (INITIAL_EPSILON - FINAL_EPSILON) / EXPLORE

        # run the selected action and observe next state and reward
        x_t1_colored, r_t, terminal = game_state.frame_step(a_t)
        x_t1 = cv2.cvtColor(cv2.resize(x_t1_colored[:,:-110,:], (84, 84)), cv2.COLOR_BGR2GRAY)
        ret, x_t1 = cv2.threshold(x_t1, 1, 255, cv2.THRESH_BINARY)
        x_t1 = np.reshape(x_t1, (84, 84, 1))
        s_t1 = np.append(x_t1, s_t[:, :, :3], axis=2)
        action_freq += a_t
        # store the transition in D
        D.append((s_t, a_t, r_t, s_t1, terminal))
        if len(D) > REPLAY_MEMORY:
            D.popleft()
        
        # only train if done observing
        if t > OBSERVE and t%train_freq==0:
            # sample a minibatch to train on
            minibatch = random.sample(D, BATCH)

            # get the batch variables
            s_j_batch = [d[0] for d in minibatch]
            a_batch = [d[1] for d in minibatch]
            r_batch = [d[2] for d in minibatch]
            s_j1_batch = [d[3] for d in minibatch]

            y_batch = []
            readout_j1_batch = readout.eval(feed_dict = {s:s_j1_batch, coeff:b_IJ2 })
            #readout_j1_batch = readout.eval(feed_dict = {s : s_j1_batch})
            for i in range(0, len(minibatch)):
                # if terminal only equals reward
                if minibatch[i][4]:
                    y_batch.append(r_batch[i])
                else:
                    y_batch.append(r_batch[i] + GAMMA * np.max(readout_j1_batch[i]))

            # perform gradient step
            train_step.run(feed_dict = {
                y : y_batch,
                a : a_batch,
                s : s_j_batch,
                coeff: b_IJ2})
            loss = cost.eval(feed_dict = {
                y : y_batch,
                a : a_batch,
                s : s_j_batch,
                coeff: b_IJ2})

        # update the old values
        s_t = s_t1
        t += 1
        if(Q_MAX < np.max(readout_t) ):
            Q_MAX = np.max(readout_t)
        # save progress every 10000 iterations
        if t % 10000 == 0:
            saver.save(sess, 'saved_networks/' + GAME + '-dqn', global_step = t)

        if(r_t==1):
            pscore += 1
        if(terminal ):#and (pscore > 5)
            print("ts", t,"Q_MAX %e" % Q_MAX,"/e", round(epsilon,3),"/pscore",pscore,"/loss",loss,"/ Q_MAX %e" % np.max(readout_t))
            pscore = 0
        if(terminal == 1):
            episode +=1
            Q_MAX = -1100000
            action_freq = np.zeros(ACTIONS)
        if(pscore > 100):
            print("Game_Ends_in Time:",int(time.time() - tick))
            break;

In [7]:
def playGame():
    tf.reset_default_graph()
    sess = tf.InteractiveSession()
    s, coeff, readout = createNetwork()
    trainNetwork(s, coeff, readout, sess)

In [ ]:
def main():
    playGame()

if __name__ == "__main__":
    tick = time.time()
    main()
    print("Game_Ends_in Time:",int(time.time() - tick))
    print("____________ END HERE _____________")

ts 49 Q_MAX 2.754931e-02 /e 0.1 /pscore 0 /loss 0 / Q_MAX 6.872475e-03
ts 99 Q_MAX 3.215498e-02 /e 0.1 /pscore 0 /loss 0 / Q_MAX 9.791390e-03
ts 149 Q_MAX 3.377236e-02 /e 0.1 /pscore 0 /loss 0 / Q_MAX 9.791390e-03
ts 199 Q_MAX 3.093128e-02 /e 0.1 /pscore 0 /loss 0 / Q_MAX 9.791390e-03
ts 249 Q_MAX 3.377236e-02 /e 0.1 /pscore 0 /loss 0 / Q_MAX 9.791390e-03
ts 299 Q_MAX 2.730861e-02 /e 0.1 /pscore 0 /loss 0 / Q_MAX -1.160236e-03
ts 349 Q_MAX 2.730861e-02 /e 0.1 /pscore 0 /loss 0 / Q_MAX -5.570401e-04
ts 399 Q_MAX 2.213255e-02 /e 0.1 /pscore 0 /loss 0 / Q_MAX 6.386475e-03
ts 449 Q_MAX 3.377236e-02 /e 0.1 /pscore 0 /loss 0 / Q_MAX 9.314194e-03
ts 499 Q_MAX 2.900385e-02 /e 0.1 /pscore 0 /loss 0 / Q_MAX -1.160236e-03
ts 549 Q_MAX 2.730861e-02 /e 0.1 /pscore 0 /loss 0 / Q_MAX -1.160236e-03
ts 599 Q_MAX 3.376120e-02 /e 0.1 /pscore 0 /loss 0 / Q_MAX 1.723048e-02
ts 649 Q_MAX 3.454043e-02 /e 0.1 /pscore 0 /loss 0 / Q_MAX 1.723048e-02
ts 699 Q_MAX 3.377236e-02 /e 0.1 /pscore 0 /loss 0 / Q_MAX 1.7

ts 5049 Q_MAX 3.455359e+00 /e 0.099 /pscore 0 /loss 0.009945767 / Q_MAX -1.078974e+00
ts 5099 Q_MAX 3.464971e+00 /e 0.099 /pscore 0 /loss 0.011308654 / Q_MAX -9.963289e-01
ts 5149 Q_MAX 3.492917e+00 /e 0.099 /pscore 0 /loss 0.013321601 / Q_MAX -3.717614e-01
ts 5199 Q_MAX 3.557001e+00 /e 0.099 /pscore 0 /loss 0.007824078 / Q_MAX -4.099500e-01
ts 5249 Q_MAX 3.620125e+00 /e 0.099 /pscore 0 /loss 0.04220805 / Q_MAX 6.195541e-03
ts 5299 Q_MAX 3.657618e+00 /e 0.099 /pscore 0 /loss 0.029065557 / Q_MAX 4.863839e-01
ts 5349 Q_MAX 3.670902e+00 /e 0.099 /pscore 0 /loss 0.010087373 / Q_MAX -6.320693e-02
ts 5399 Q_MAX 3.653121e+00 /e 0.099 /pscore 0 /loss 0.044541325 / Q_MAX -2.515608e-01
ts 5449 Q_MAX 3.627898e+00 /e 0.099 /pscore 0 /loss 0.024867032 / Q_MAX -8.318756e-01
ts 5499 Q_MAX 3.632617e+00 /e 0.099 /pscore 0 /loss 0.039103016 / Q_MAX -1.055193e+00
ts 5549 Q_MAX 3.644709e+00 /e 0.098 /pscore 0 /loss 0.006952888 / Q_MAX -1.516496e+00
ts 5599 Q_MAX 3.648593e+00 /e 0.098 /pscore 0 /loss 0.012

ts 10162 Q_MAX 3.943110e+00 /e 0.097 /pscore 0 /loss 0.030505905 / Q_MAX 2.944385e+00
ts 10225 Q_MAX 4.011993e+00 /e 0.097 /pscore 1 /loss 0.02401191 / Q_MAX 3.332337e+00
ts 10275 Q_MAX 4.118303e+00 /e 0.097 /pscore 0 /loss 0.059307203 / Q_MAX -3.575999e-01
ts 10325 Q_MAX 4.106941e+00 /e 0.097 /pscore 0 /loss 0.06605996 / Q_MAX -7.352801e-02
ts 10375 Q_MAX 4.094774e+00 /e 0.097 /pscore 0 /loss 0.07864037 / Q_MAX 6.552709e-02
ts 10427 Q_MAX 4.048358e+00 /e 0.097 /pscore 0 /loss 0.1008577 / Q_MAX 3.188184e+00
ts 10477 Q_MAX 4.011389e+00 /e 0.097 /pscore 0 /loss 0.07142186 / Q_MAX -9.036620e-02
ts 10527 Q_MAX 4.126570e+00 /e 0.097 /pscore 0 /loss 0.07887372 / Q_MAX -4.945024e-01
ts 10577 Q_MAX 4.146118e+00 /e 0.097 /pscore 0 /loss 0.045829765 / Q_MAX -6.545394e-01
ts 10642 Q_MAX 4.082061e+00 /e 0.097 /pscore 1 /loss 0.029651888 / Q_MAX 7.461581e-01
ts 10745 Q_MAX 3.982712e+00 /e 0.097 /pscore 2 /loss 0.03704769 / Q_MAX -1.760421e-01
ts 10811 Q_MAX 4.015598e+00 /e 0.097 /pscore 1 /loss 0.0

ts 16652 Q_MAX 4.659424e+00 /e 0.095 /pscore 1 /loss 0.6650683 / Q_MAX -8.201501e-01
ts 16702 Q_MAX 4.637483e+00 /e 0.095 /pscore 0 /loss 0.022263497 / Q_MAX -1.304953e+00
ts 16760 Q_MAX 4.828920e+00 /e 0.095 /pscore 0 /loss 0.051248193 / Q_MAX 2.371652e+00
ts 16883 Q_MAX 4.849734e+00 /e 0.095 /pscore 2 /loss 0.049312633 / Q_MAX -3.954404e-01
ts 17018 Q_MAX 4.876776e+00 /e 0.095 /pscore 2 /loss 0.047557674 / Q_MAX 2.120920e+00
ts 17184 Q_MAX 4.891265e+00 /e 0.095 /pscore 3 /loss 0.025841769 / Q_MAX 2.930343e+00
ts 17234 Q_MAX 4.838834e+00 /e 0.095 /pscore 0 /loss 0.119833164 / Q_MAX -5.977563e-01
ts 17284 Q_MAX 4.838996e+00 /e 0.095 /pscore 0 /loss 0.04190179 / Q_MAX -6.155547e-01
ts 17339 Q_MAX 4.819141e+00 /e 0.095 /pscore 0 /loss 0.07568035 / Q_MAX 1.817162e+00
ts 17389 Q_MAX 4.774769e+00 /e 0.095 /pscore 0 /loss 0.04550145 / Q_MAX 1.919589e-01
ts 17454 Q_MAX 4.787069e+00 /e 0.095 /pscore 1 /loss 0.015598 / Q_MAX 2.489298e-01
ts 17577 Q_MAX 4.783680e+00 /e 0.094 /pscore 2 /loss 0.08

ts 24909 Q_MAX 6.222136e+00 /e 0.092 /pscore 2 /loss 0.043590344 / Q_MAX -6.571806e-01
ts 24959 Q_MAX 6.251303e+00 /e 0.092 /pscore 0 /loss 0.12666741 / Q_MAX -7.708218e-01
ts 25023 Q_MAX 6.287064e+00 /e 0.092 /pscore 1 /loss 0.06532781 / Q_MAX 2.334727e+00
ts 25294 Q_MAX 6.301522e+00 /e 0.092 /pscore 6 /loss 0.022299064 / Q_MAX 7.096135e-01
ts 25348 Q_MAX 6.438194e+00 /e 0.092 /pscore 0 /loss 0.04980334 / Q_MAX -2.424995e-01
ts 25398 Q_MAX 6.532208e+00 /e 0.092 /pscore 0 /loss 0.07618924 / Q_MAX -1.486497e+00
ts 25558 Q_MAX 6.535686e+00 /e 0.092 /pscore 3 /loss 0.030406266 / Q_MAX -1.563523e+00
ts 25608 Q_MAX 6.245888e+00 /e 0.092 /pscore 0 /loss 0.110233955 / Q_MAX -1.419137e+00
ts 25694 Q_MAX 6.412474e+00 /e 0.092 /pscore 1 /loss 0.061053474 / Q_MAX -9.131844e-01
ts 25744 Q_MAX 6.584114e+00 /e 0.092 /pscore 0 /loss 0.1350022 / Q_MAX -7.808521e-01
ts 25832 Q_MAX 6.544469e+00 /e 0.092 /pscore 1 /loss 0.09239416 / Q_MAX -4.688419e-01
ts 25960 Q_MAX 6.637707e+00 /e 0.092 /pscore 2 /loss

...

ts 127035 Q_MAX 1.518060e+01 /e 0.058 /pscore 1 /loss 0.6561615 / Q_MAX 7.104313e+00
ts 127177 Q_MAX 1.476667e+01 /e 0.058 /pscore 3 /loss 0.46173868 / Q_MAX 5.528726e+00
ts 127264 Q_MAX 1.453069e+01 /e 0.058 /pscore 1 /loss 0.2660584 / Q_MAX -4.046440e+00
ts 127314 Q_MAX 1.438717e+01 /e 0.058 /pscore 0 /loss 0.20226848 / Q_MAX -2.463182e+00
ts 127364 Q_MAX 1.442167e+01 /e 0.058 /pscore 0 /loss 0.087924205 / Q_MAX -3.262127e+00
ts 127490 Q_MAX 1.471266e+01 /e 0.058 /pscore 2 /loss 0.04677709 / Q_MAX -1.344944e+00
ts 127540 Q_MAX 1.451965e+01 /e 0.058 /pscore 0 /loss 0.354196 / Q_MAX -2.183629e+00
ts 127850 Q_MAX 1.566566e+01 /e 0.058 /pscore 7 /loss 0.042299002 / Q_MAX -1.536343e+00
ts 128128 Q_MAX 1.553963e+01 /e 0.058 /pscore 6 /loss 0.09022654 / Q_MAX -1.049651e+00
ts 128198 Q_MAX 1.540086e+01 /e 0.058 /pscore 1 /loss 0.15703633 / Q_MAX 5.106414e+00
ts 128301 Q_MAX 1.517118e+01 /e 0.058 /pscore 2 /loss 0.14442123 / Q_MAX -2.482619e-01
ts 128351 Q_MAX 1.443543e+01 /e 0.058 /pscore 0 

ts 141604 Q_MAX 1.559304e+01 /e 0.053 /pscore 6 /loss 0.20327209 / Q_MAX 6.059899e-01
ts 141777 Q_MAX 1.588928e+01 /e 0.053 /pscore 4 /loss 0.07883769 / Q_MAX 1.035170e+00
ts 141864 Q_MAX 1.532514e+01 /e 0.053 /pscore 1 /loss 0.16611852 / Q_MAX -1.684906e+00
ts 142008 Q_MAX 1.545906e+01 /e 0.053 /pscore 3 /loss 0.049346525 / Q_MAX 1.272255e+01
ts 142058 Q_MAX 1.449243e+01 /e 0.053 /pscore 0 /loss 0.078175716 / Q_MAX -1.269380e+00
ts 142145 Q_MAX 1.500287e+01 /e 0.053 /pscore 1 /loss 0.05815638 / Q_MAX 6.321859e-01
ts 142351 Q_MAX 1.525471e+01 /e 0.053 /pscore 4 /loss 0.043238834 / Q_MAX 1.950042e-01
ts 142401 Q_MAX 1.453982e+01 /e 0.053 /pscore 0 /loss 0.21375765 / Q_MAX -3.265015e-01
ts 142578 Q_MAX 1.523759e+01 /e 0.053 /pscore 4 /loss 0.028718447 / Q_MAX 1.449102e+00
ts 142666 Q_MAX 1.483949e+01 /e 0.053 /pscore 1 /loss 0.51642096 / Q_MAX -1.438500e+00
ts 142717 Q_MAX 1.473965e+01 /e 0.053 /pscore 0 /loss 0.14491023 / Q_MAX -3.278526e+00
ts 143173 Q_MAX 1.541109e+01 /e 0.053 /pscore

ts 159847 Q_MAX 1.547355e+01 /e 0.047 /pscore 9 /loss 0.24831347 / Q_MAX -1.374554e+00
ts 160229 Q_MAX 1.567593e+01 /e 0.047 /pscore 9 /loss 0.023068883 / Q_MAX -5.063105e-01
ts 160649 Q_MAX 1.541363e+01 /e 0.047 /pscore 10 /loss 0.060612477 / Q_MAX 1.318349e+01
ts 160773 Q_MAX 1.525526e+01 /e 0.047 /pscore 2 /loss 0.16181293 / Q_MAX -2.357622e+00
ts 161341 Q_MAX 1.582168e+01 /e 0.047 /pscore 14 /loss 0.08686195 / Q_MAX 1.273988e-01
ts 161621 Q_MAX 1.539091e+01 /e 0.047 /pscore 6 /loss 0.8005007 / Q_MAX -1.041413e+00
ts 161867 Q_MAX 1.588989e+01 /e 0.046 /pscore 6 /loss 0.0424496 / Q_MAX 9.753169e+00
ts 162066 Q_MAX 1.626027e+01 /e 0.046 /pscore 4 /loss 0.09376745 / Q_MAX -7.527614e-01
ts 162116 Q_MAX 1.528175e+01 /e 0.046 /pscore 0 /loss 0.16605428 / Q_MAX 2.405058e-01
ts 162331 Q_MAX 1.605902e+01 /e 0.046 /pscore 5 /loss 0.05162428 / Q_MAX -8.660842e-01
ts 162537 Q_MAX 1.600248e+01 /e 0.046 /pscore 4 /loss 0.041783884 / Q_MAX 5.563502e+00
ts 162587 Q_MAX 1.468676e+01 /e 0.046 /pscore

ts 177287 Q_MAX 1.589802e+01 /e 0.041 /pscore 5 /loss 0.79487073 / Q_MAX -7.170035e-01
ts 177375 Q_MAX 1.535454e+01 /e 0.041 /pscore 1 /loss 0.0401121 / Q_MAX 1.704739e+00
ts 177572 Q_MAX 1.589357e+01 /e 0.041 /pscore 4 /loss 0.11088631 / Q_MAX 2.077514e+00
ts 177852 Q_MAX 1.560678e+01 /e 0.041 /pscore 6 /loss 0.030840028 / Q_MAX -3.290310e-01
ts 177907 Q_MAX 1.478339e+01 /e 0.041 /pscore 0 /loss 0.09559822 / Q_MAX -9.992980e-01
ts 177998 Q_MAX 1.531286e+01 /e 0.041 /pscore 1 /loss 0.17679779 / Q_MAX -1.072104e-01
ts 178058 Q_MAX 1.506371e+01 /e 0.041 /pscore 0 /loss 0.06797167 / Q_MAX 4.711291e-01
ts 178521 Q_MAX 1.576339e+01 /e 0.041 /pscore 11 /loss 0.7826299 / Q_MAX 7.508596e+00
ts 178681 Q_MAX 1.579002e+01 /e 0.041 /pscore 3 /loss 0.022819538 / Q_MAX -1.177876e-01
ts 178894 Q_MAX 1.589257e+01 /e 0.041 /pscore 5 /loss 0.088394865 / Q_MAX 5.419772e+00
ts 179182 Q_MAX 1.601688e+01 /e 0.041 /pscore 7 /loss 0.046177752 / Q_MAX 1.171161e+00
ts 179232 Q_MAX 1.541904e+01 /e 0.041 /pscore 

ts 195711 Q_MAX 1.517951e+01 /e 0.035 /pscore 0 /loss 0.06195602 / Q_MAX 2.421820e+00
ts 195838 Q_MAX 1.588629e+01 /e 0.035 /pscore 2 /loss 0.15397617 / Q_MAX -2.241561e+00
ts 195899 Q_MAX 1.531793e+01 /e 0.035 /pscore 0 /loss 0.10410221 / Q_MAX -4.343447e+00
ts 195949 Q_MAX 1.480137e+01 /e 0.035 /pscore 0 /loss 0.86723524 / Q_MAX -1.122292e+00
ts 196019 Q_MAX 1.511024e+01 /e 0.035 /pscore 1 /loss 0.10063694 / Q_MAX 2.020050e+00
ts 196524 Q_MAX 1.624621e+01 /e 0.035 /pscore 12 /loss 0.034320652 / Q_MAX 1.666007e-01
ts 196722 Q_MAX 1.616612e+01 /e 0.035 /pscore 4 /loss 0.33602315 / Q_MAX 2.410019e+00
ts 196779 Q_MAX 1.586959e+01 /e 0.035 /pscore 0 /loss 0.060218867 / Q_MAX 2.009006e+00
ts 196838 Q_MAX 1.562627e+01 /e 0.035 /pscore 0 /loss 0.04000424 / Q_MAX -2.151591e-02
ts 197776 Q_MAX 1.605970e+01 /e 0.034 /pscore 24 /loss 0.060106415 / Q_MAX 5.363568e-01
ts 197862 Q_MAX 1.546391e+01 /e 0.034 /pscore 1 /loss 0.021992855 / Q_MAX -9.858126e-01
ts 197948 Q_MAX 1.561350e+01 /e 0.034 /psco

In [ ]:
Jai Gurudev